# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [12]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [13]:
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.openai_like import OpenAILike

Settings.llm = OpenAILike(
                          temperature=0.7,
                          model= "gpt-3.5-turbo", #   "gemma-3-27b-it",                      # Can be any name, not necessarily openai's gpt
                          api_base="http://127.0.0.1:8080/v1",
                          api_key="fake",
                          timeout= 120.0,
                          is_chat_model = True,
                          is_function_calling_model=True,
                          )


Settings.embed_model = OllamaEmbedding(
                                        model_name="nomic-embed-text",      # Using foundational model may be overkill
                                        base_url="http://localhost:11434",
                                       )

In [14]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["/home/ashok/Documents/csvrag/metagpt.pdf"]).load_data()

In [15]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

## Setup the Query Tools

In [16]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(
                                 nodes,
                                 show_progres = True
                               )

In [17]:
summary_query_engine = summary_index.as_query_engine(
                                                     response_mode="tree_summarize",
                                                     use_async=True,  # Optional but gives faster response
                                                    )

vector_query_engine = vector_index.as_query_engine(show_progress = True)

In [18]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
                                             query_engine=summary_query_engine,
                                             description=(
                                                          "Useful for summarization questions related to MetaGPT"
                                                          ),
                                              )

vector_tool = QueryEngineTool.from_defaults(
                                             query_engine=vector_query_engine,
                                             description=(
                                                           "Useful for retrieving specific context from the MetaGPT paper."
                                                         ),
                                            )

In [19]:
#from utils import get_doc_tools

#vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [20]:
llm = Settings.llm

In [21]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "Agent Roles and Communication in MetaGPT"}


In [ ]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

In [ ]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

In [ ]:
response = agent.chat("Tell me the results over one of the above datasets.")

## Lower-Level: Debuggability and Control

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [ ]:
step_output = agent.run_step(task.task_id)

In [ ]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

In [ ]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

In [ ]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

In [ ]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(str(response))